# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [2]:
!pip install openai
from openai import OpenAI
import os

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00


In [ ]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [3]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [4]:
#Functio to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [23]:
#Definition of the tables.
import pandas as pd

# Table and definitions sample
data = {'table': ['hotel', 'guests', 'incidents', 'reservations', 'stays'],
        'definition': ['Information about the hotel',
                       'Guests of the hotel',
                       'Incidents created by guests',
                       'Reservations of guests',
                       'Stays are used to combine Reservations, Guests and Hotels']}
df = pd.DataFrame(data)
df

,table,definition
0,hotel,Information about the hotel
1,guests,Guests of the hotel
2,incidents,Incidents created by guests
3,reservations,Reservations of guests
4,stays,"Stays are used to combine Reservations, Guests..."


In [14]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [15]:
print(text_tables)

hotel: Information about the hotel
guests: Guests of the hotel
incidents: Incidents created by guests
reservations: Reservations of guests
stays: Stays are used to combine Reservations, Guests and Hotels


In [16]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
{question}
"""


In [17]:
#Creating the prompt, with the user questions and the tables definitions.
pqt1 = prompt_question_tables.format(tables=text_tables, question="Get Incidents of a specific hotel")

In [18]:
print(return_OAI(pqt1))

```json
{
    "tables": ["hotel", "incidents"]
}
```


In [19]:
pqt3 = prompt_question_tables.format(tables=text_tables,
                                     question="Return guests which are older then 35")

In [12]:
print(return_OAI(pqt3))

```json
{
    "tables": ["guests"]
}
```


# Exercise
 - Complete the prompts similar to what we did in class.
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [20]:
question = prompt_question_tables.format(tables=text_tables, question="Select all guests which were more then 5 times in a specific Hotel")
print(return_OAI(question))

```json
{
    "tables": ["guests", "stays"]
}
```


In [21]:
question = prompt_question_tables.format(tables=text_tables, question="Select all hotels with checked-out reservations")
print(return_OAI(question))

```json
{
    "tables": ["hotel", "reservations", "stays"]
}
```


In [22]:
question = prompt_question_tables.format(tables=text_tables, question="Return a sample Incident of a specific guest")
print(return_OAI(question))

```json
{
    "tables": ["incidents", "guests"]
}
```
